## 1. 모듈삽입

In [2]:
import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
# from tensorflow.python.framework import ops as tf_ops

# import keras.backend.tensorflow_backend as K

C:\Users\Jisoo\Anaconda3\envs\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jisoo\Anaconda3\envs\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jisoo\Anaconda3\envs\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jisoo\Anaconda3\envs\venv\lib\site-packages\tensorflow\python\frame

## 2. 데이터 설명 파일 불러오기

In [3]:
# 데이터 프레임 불러오기

total_data_final_2 = pd.read_csv('./total_data_final_2.csv')

In [4]:
# 필요한 부분의 data frame 설정

total_data_final_2.columns
df = total_data_final_2[['folder_name','image_num','category_label','category_name',
                         'category_type','texture','shape','style']]
df

,folder_name,image_num,category_label,category_name,category_type,texture,shape,style
0,Sheer_Pleated-Front_Blouse,img_00000001.jpg,3,Blouse,1,NaN,shift,NaN
1,Sheer_Pleated-Front_Blouse,img_00000002.jpg,3,Blouse,1,NaN,shift,NaN
2,Sheer_Pleated-Front_Blouse,img_00000003.jpg,3,Blouse,1,NaN,slip,NaN
3,Sheer_Pleated-Front_Blouse,img_00000004.jpg,3,Blouse,1,NaN,NaN,NaN
4,Sheer_Pleated-Front_Blouse,img_00000005.jpg,3,Blouse,1,NaN,NaN,shark
...,...,...,...,...,...,...,...,...
289103,Paisley_Print_Babydoll_Dress,img_00000050.jpg,41,Dress,3,palm springs,print shift,NaN
289104,Paisley_Print_Babydoll_Dress,img_00000051.jpg,41,Dress,3,palm,NaN,NaN
289105,Paisley_Print_Babydoll_Dress,img_00000052.jpg,41,Dress,3,palm,NaN,NaN
289106,Paisley_Print_Babydoll_Dress,img_00000053.jpg,41,Dress,3,palm springs,print shift,NaN


## 3. 이미지 데이터 불러오기

In [ ]:
# 이미지를 불러오는 방법 2가지

# 1. plt 사용
# plt.imread('파일이름.jpg') : 숫자 array로 불러옴
# plt.imshow : 위의 숫자를 이미지로 보여줌

# 2. pillow 사용
# Image.open('파일이름.jpg') : 이미지로 불러옴
# np.array(위의 이미지) : 위의 이미지를 숫자로 보여줌

In [7]:
# 우선 예시로 이미지 열어봄

mm = Image.open('E:/Project/img/Sheer_Pleated-Front_Blouse/img_00000002.jpg')
print(mm.size) 
print(mm.format)
# np.array(mm)

# 이미지 사이즈 다 달라서 학습시 동일하게 맞춰주는 작업 필요!!

(205, 300)
JPEG


## 4. texture 분류모델 생성: tensorflow

### 4.1. 데이터 준비

In [8]:
# 이미지 데이터 가져오기 위해 사용할 이미지에 대한 설명 df 준비
# 불필요 column, NA 값 row, category_type=3 인 값 삭제
# 64000 row 남음


# 불필요 col 삭제
df_t = df.drop(['style','shape'], axis=1)

# texture NA 삭제
df_t = df_t.dropna(axis=0, how='any')
df_t.shape #100000

# category_type = 3 삭제
df_t = df_t[df_t.category_type != 3]
df_t.shape

# dropna 하면서 drop된 행의 index 비어버림 : index 초기화
df_t = df_t.reset_index(drop=True)
df_t

,folder_name,image_num,category_label,category_name,category_type,texture
0,Sheer_Pleated-Front_Blouse,img_00000009.jpg,3,Blouse,1,southwestern-patterned
1,Sheer_Pleated-Front_Blouse,img_00000035.jpg,3,Blouse,1,polka dot
2,Sheer_Pleated-Front_Blouse,img_00000036.jpg,3,Blouse,1,tribal
3,Sheer_Pleated-Front_Blouse,img_00000038.jpg,3,Blouse,1,chiffon floral
4,Sheer_Pleated-Front_Blouse,img_00000040.jpg,3,Blouse,1,zig
...,...,...,...,...,...,...
64746,Destroyed_Denim_Cutoffs,img_00000015.jpg,24,Cutoffs,2,diamond
64747,Destroyed_Denim_Cutoffs,img_00000016.jpg,24,Cutoffs,2,diamond print
64748,Destroyed_Denim_Cutoffs,img_00000017.jpg,24,Cutoffs,2,diamond print
64749,Destroyed_Denim_Cutoffs,img_00000018.jpg,24,Cutoffs,2,paint


### 4.2. 데이터 확보: X_data

In [ ]:
# 실제 학습에 사용할 이미지 데이터를 픽셀화 시킨 데이터 준비

# X에 해당되는 데이터를 모아보자 : 이미지 픽셀 데이터 
# X : 이미지 픽셀 값
# y : texture 이름 one hot encofing 
# 사용할 df: df_t


img_w = 200
img_h = 200

X_data = []

for _ in range(df_t.shape[0]):
    
    img_dir = df_t['folder_name'][_] 
    img_name = df_t['image_num'][_] 
    classes = df_t['texture'].unique()
    full_dir = str('E:/Project/img/' + img_dir + '/' + img_name) #이미지 주소
    
    img = Image.open(full_dir)
    #img = img.convert("RGB")
    img = img.resize((img_w, img_h))
    data = np.array(img)
    
    X_data.append(data)
    
    if _ % 1000 == 0:
        print(_)    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000


In [ ]:
X_data = np.array(X_data)
np.save('texture_X_data.npy', X_data)

### 4.3 데이터 확보: Y_data

In [13]:
# y에 해당되는 데이터를 모아보자 : texture 에 대한 값, one-hot encoding 필요
# 사용할 df = df_t


y_data = df['texture']
Y_data = pd.get_dummies(y_data, columns=['texture'])
Y_data



,abstract chevron,abstract diamond,abstract floral print,abstract geo,abstract paisley,abstract print,abstract printed,abstract stripe,animal,animal print,...,stripes,structured,tonal,tribal,tribal-inspired,two-tone,varsity-striped,watercolor,zig,zigzag
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64746,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64747,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64748,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64749,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 4.4. 데이터 분리 및 전처리

In [17]:
# 1. np.array 변환
X_data = np.array(X_data)
Y_data = np.array(Y_data)

In [18]:
# 2. 정규화
# 픽셀값은 0~255 사이의 값이기에 255로 나눠서 정규화 진행
X_data = X_data.astype(float) / 255
Y_data = Y_data.astype(float) / 255

In [23]:
# 저장
XY = (X_data, Y_data)
np.save("texture_data", XY)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-38120a38659d>", line 3, in <module>
    np.save('texture_data.npy', XY)
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\numpy\lib\npyio.py", line 527, in save
    arr = np.asanyarray(arr)
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\numpy\core\numeric.py", line 591, in asanyarray
    return array(a, dtype, copy=False, order=order, subok=True)
ValueError: could not broadcast input array from shape (64751,64,64,3) into shape (64751)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 

ValueError: could not broadcast input array from shape (64751,64,64,3) into shape (64751)

In [ ]:
# 3. train, test 구분
X_train, X_test, Y_train, Y_test = train_test_split(X_data,Y_data)
xy = (X_train, X_test, Y_train, Y_test)

# 저장
np.save("texture_data_split.npy", xy)  

In [20]:
np.load('texture_data_split.npy')

array([array([[[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        ...,

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
 

In [149]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [151]:
print(X_train.shape)
print(X_train.shape[0])

(48563, 64, 64, 3)
48563


In [153]:
categories = t_y_data.columns
nb_classes = len(categories)

In [154]:
#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255